<a href="https://colab.research.google.com/github/Visionaryofthefuture/Machine-Learning/blob/main/NLP_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!wget https://www.kaggle.com/c/fake-news/data?select=train.csv


In [ ]:
import pandas as pd
df = pd.read_csv("/content/train.csv", on_bad_lines = "skip", engine = "python")
print(df.shape)
df

(21035, 5)


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
21030,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
21031,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
21032,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
21033,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
for i in df.columns:
  print(f" {i} : {df[i].isnull().sum()}")

 id : 0
 title : 572
 author : 1991
 text : 59
 label : 22


In [ ]:
df = df.dropna()

In [ ]:
for i in df.columns:
  print(f" {i} : {df[i].isnull().sum()}")

 id : 0
 title : 0
 author : 0
 text : 0
 label : 0


In [ ]:
y = df["label"]
df.drop(columns = ["id", "label"], inplace = True)
X = df


<ipython-input-100-b9fefd606548>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ["id", "label"], inplace = True)


In [ ]:
y = [value for value in y if value.isdigit()]


In [ ]:
X

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...
21030,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
21031,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
21032,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
21033,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
import nltk
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Embedding
import re
from nltk.corpus import stopwords

In [ ]:
vocab = 5000
messages = X.copy()
messages

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...
21030,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
21031,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
21032,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
21033,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(messages.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', messages.iloc[i]['title'])
    review = review.lower()
    review = review.split()

    review =[words for words in review if not words in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [ ]:
onehot = [one_hot(words, vocab) for words in corpus]

In [ ]:
sent_length = 20
embedded  = pad_sequences(onehot, padding = 'pre', maxlen = sent_length)
embedded

array([[   0,    0,    0, ..., 2707, 4758, 4635],
       [   0,    0,    0, ..., 2621,  743, 1182],
       [   0,    0,    0, ..., 2389, 3065,  565],
       ...,
       [   0,    0,    0, ..., 4570, 4310, 1929],
       [   0,    0,    0, ..., 4379, 3662, 4969],
       [   0,    0,    0, ..., 4344, 1136, 4468]], dtype=int32)

In [ ]:
embedding_features = 40
model = Sequential()
model.add(Embedding(vocab, embedding_features,input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', metrics =['accuracy'], optimizer = 'adam')

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_5 (LSTM)               (None, 100)               56400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import numpy as np
X_train = np.array(embedded)
Y_train = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.3, random_state = 0)
type(Y_train[0])

ValueError: Found input variables with inconsistent numbers of samples: [18475, 18473]

`

[link text](https://)model.fit(X_train, Y_train,validation_data = (X_test, Y_test), epochs = 40, batch_size =20)

In [ ]:
model.fit(X_train, Y_train,validation_data = (X_test, Y_test), epochs = 40, batch_size =20)